# Plotting of chemcials production emissions in interactive dashboard

In [1]:
## Packages and data import
import numpy as np
import pandas as pd
from tqdm import tqdm

def filter_df(df, cols, filters):
    for col, filt in zip(cols, filters):
        df = df[[i in filt for i in df[col]]]
    return df

In [3]:
data_path = '../data/'
country_groupings = pd.read_csv(data_path+'extra_inputs/country_regions.csv')

In [37]:
country_groupings['COUNTRY/TERRITORY'] = country_groupings['COUNTRY/TERRITORY'].str.title()
product_grouping = pd.read_csv(data_path+'extra_inputs/product_groups.csv', index_col=0)
product_grouping['PRODUCT'] = product_grouping['PRODUCT'].str.title()

In [38]:
product_grouping['Product group'].unique()

array(['Organic chemicals', 'Thermoplastics',
       'Thermosets, fibre & elastomers', nan, 'Primary chemicals',
       'Solvents, additives & explosives', 'Other downstream',
       'Other intermediates', 'N-fertilisers'], dtype=object)

In [50]:
print(list((filter_df(product_grouping, ['Product group'], [['Other downstream']]).sort_values('PRODUCT')['PRODUCT'])))

['Dmt', 'Eva Copolymers', 'Nb Copolymers', 'Sb Latices']


In [3]:
# Import data and correct country names for Power BI display
data_path = '../data/'
facility_emissions_file = data_path+'combined/icisFacilityEmissions_ihsWeighted_w_uncertainties_allgases_amm.parquet'

ifa_emissions_file = data_path+'combined/ifaProductionEmissions.csv'

country_replace_dict = {'United States': 'USA', 'Taiwan, Province Of China':'Taiwan', 'Netherlands-The':'Netherlands', 'Russian Federation':'Russia', 'Hong Kong Special Administrative Region':'Hong Kong', 'Dominican Republ.':'Dominican Republic', 'Congo (Dem.Rep.)':'Democratic Republic of the Congo', 'Congo Republic':'Republic of the Congo', 'Slovak Republic':'Slovakia', 'Serbia/Montenegro/Kosovo':'Serbia', 'Bosnia/Herzegovina':'Bosnia and Herzegovina', 'Myanmar(Burma)':'Myanmar', 'Papua N.G.':'Papua New Guinea'}# 'Congo (Dem.Rep.)':'COD', 'Congo Republic':'COG', 'Vietnam':'VNM'})

facility_emissions = pd.read_parquet(facility_emissions_file)

# Load country groupings and join
country_groupings = pd.read_csv(data_path+'extra_inputs/country_regions.csv')
facility_emissions = pd.merge(facility_emissions, country_groupings, on='COUNTRY/TERRITORY', how='left')

facility_emissions['COUNTRY/TERRITORY'] = facility_emissions['COUNTRY/TERRITORY'].str.title().replace(country_replace_dict)
#facility_emissions['Type'] = facility_emissions['Type'].replace({'IHS CtOG': 'Cradle-to-out-gate', 'Raw Material':'Feedstock'})

In [4]:
# np.concatenate((facility_emissions.sort_values('COUNTRY/TERRITORY')['COUNTRY/TERRITORY'].unique().reshape(-1,1), country_groupings.sort_values('COUNTRY/TERRITORY')['COUNTRY/TERRITORY'].unique().reshape(-1,1)), axis=-1)

In [5]:
# When no coordinates for plant - use country average
facility_emissions[['LATITUDE','LONGITUDE']] = facility_emissions[['LATITUDE','LONGITUDE']].replace({'n.a.':0}).astype(float)

avg_coords = facility_emissions[facility_emissions['LATITUDE']!=0].groupby(['COUNTRY/TERRITORY']).mean()[['LATITUDE','LONGITUDE']]
fixed_zeros = facility_emissions[facility_emissions['LATITUDE']==0].merge(avg_coords, on=['COUNTRY/TERRITORY'], how='left')
fixed_zeros['LATITUDE'], fixed_zeros['LONGITUDE'] = fixed_zeros['LATITUDE_y'], fixed_zeros['LONGITUDE_y']

corrected_emissions = pd.concat((facility_emissions[facility_emissions['LATITUDE']!=0], fixed_zeros.drop(columns=['LATITUDE_x','LONGITUDE_x','LATITUDE_y','LONGITUDE_y']))).sort_values(['PRODUCT', 'COUNTRY/TERRITORY', 'STATE', 'COMPANY', 'SITE', '#', 'ROUTE']).reset_index(drop=True)

corrected_emissions.head()

,PRODUCT,COUNTRY/TERRITORY,STATE,COMPANY,SITE,#,ROUTE,TECHNOLOGY,LICENSOR,START_YR,...,2042_sigma,2043_sigma,2044_sigma,2045_sigma,2046_sigma,2047_sigma,2048_sigma,2049_sigma,2050_sigma,Region
0,"1,4-BUTANEDIOL",Belgium,n.a.,BASF,FELUY,1.0,MALEIC ANHYDRIDE,HYDRATION,KVAERNER PROCESS TECH,1997.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Europe
1,"1,4-BUTANEDIOL",Belgium,n.a.,BASF,FELUY,1.0,MALEIC ANHYDRIDE,HYDRATION,KVAERNER PROCESS TECH,1997.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Europe
2,"1,4-BUTANEDIOL",Belgium,n.a.,BASF,FELUY,1.0,MALEIC ANHYDRIDE,HYDRATION,KVAERNER PROCESS TECH,1997.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Europe
3,"1,4-BUTANEDIOL",Belgium,n.a.,BASF,FELUY,1.0,MALEIC ANHYDRIDE,HYDRATION,KVAERNER PROCESS TECH,1997.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Europe
4,"1,4-BUTANEDIOL",Belgium,n.a.,BASF,FELUY,1.0,MALEIC ANHYDRIDE,HYDRATION,KVAERNER PROCESS TECH,1997.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Europe


In [6]:
# filt = filter_df(facility_emissions, ['Gas', 'Type'], [['CO2e_100a'], ['Raw Material', 'Feedstock','Organic chemicals', 'Primary chemicals', 'Other intermediates', 'Direct Utilities', 'Indirect Utilities', 'Direct Process']])

In [7]:
# Add product grouping and save facility specific
product_grouping = pd.read_csv(data_path+'extra_inputs/product_groups.csv', index_col=0)
product_grouping = product_grouping.dropna(subset=['Product group']).reset_index(drop=True)
grouped_emissions = pd.merge(facility_emissions, product_grouping, on='PRODUCT', how='left')

group_types = {'Other downstream': 'Downstream', 'Other intermediates': 'Intermediates', 'Primary chemicals':'Intermediates', 'Solvents, additives & explosives': 'Downstream', 'Thermoplastics': 'Downstream', 'Organic chemicals':'Intermediates', 'Thermosets, fibre & elastomers':'Downstream', 'N-fertilisers':'Downstream'}
grouped_emissions['Product type'] = grouped_emissions['Product group'].replace(group_types)
grouped_emissions['Type'] = grouped_emissions['Type'].replace({'Indirect Utilities':'Indirect Energy Use', 'Direct Utilities':'Direct Energy Use'})
grouped_emissions.to_parquet('C:/Users\lukec\OneDrive - University of Cambridge\PhD\Power_BI\input_data/facility_emissions_amm.parquet')

In [8]:
## Group by country

del facility_emissions

grouped = grouped_emissions.groupby(['Region', 'COUNTRY/TERRITORY', 'Product group', 'PRODUCT', 'Gas', 'Type']).sum()
grouped = grouped.reset_index()

grouped.to_parquet('C:/Users\lukec\PycharmProjects\petrochemical-data\data\combined/grouped_amm.parquet')

In [9]:
## Reduce facility specific for use on dashboard

# grouped_emissions = pd.read_parquet('C:/Users\lukec\OneDrive - University of Cambridge\PhD\Power_BI\input_data/facility_emissions.parquet')
# filtered = filter_df(grouped_emissions, ['Gas'], [['CO2e_100a', 'CO2e_20a', 'Carbon dioxide', 'Carbon monoxide', 'Methane', 'Nitric oxide']])
# filtered = filtered[list(filtered.columns[:15])+['Region', 'Product group', 'Product type', '2000', '2010', '2020','2030', '2040', '2050', '2000_sigma', '2010_sigma', '2020_sigma','2030_sigma', '2040_sigma', '2050_sigma']]
# filtered.to_parquet('C:/Users\lukec\OneDrive - University of Cambridge\PhD\Power_BI\input_data/facility_emissions_reduced.parquet')

In [10]:
# Add product grouping
# product_grouping = pd.read_csv(data_path+'extra_inputs/product_groups.csv', index_col=0)
# product_grouping = product_grouping.dropna(subset=['Product group']).reset_index(drop=True)
# grouped_emissions = pd.merge(facility_emissions, product_grouping, on='PRODUCT', how='left')
#grouped_emissions = pd.merge(corrected_emissions, product_grouping, on='PRODUCT', how='left')

# clear space in memory

#grouped = grouped.drop(columns=['LATITUDE', 'LONGITUDE']).reset_index()

#grouped['Fertiliser type'] = ['Fertiliser' if i == 'N-fertilisers' else 'Non-fertiliser' for i in grouped['Product group']]

In [2]:
grouped = pd.read_parquet('C:/Users\lukec\PycharmProjects\petrochemical-data\data\combined/grouped_amm.parquet')

group_types = {'Other downstream': 'Downstream', 'Other intermediates': 'Intermediates', 'Primary chemicals':'Intermediates', 'Solvents, additives & explosives': 'Downstream', 'Thermoplastics': 'Downstream', 'Organic chemicals':'Intermediates', 'Thermosets, fibre & elastomers':'Downstream', 'N-fertilisers':'Downstream'}
grouped['Product type'] = grouped['Product group'].replace(group_types)
grouped = grouped.drop(columns=['LATITUDE', 'LONGITUDE'])

In [3]:
# Even out intermediate feedstocks to raw materials total
rm_groups = grouped[grouped['Type']=='Raw Material']
int_groups = grouped[grouped['Type'].isin(['Feedstock', 'Organic chemicals', 'Primary chemicals', 'Other intermediates'])]
int_groups_tot = int_groups.groupby(['Product type', 'Region', 'COUNTRY/TERRITORY', 'Product group', 'PRODUCT', 'Gas']).sum()
merged = int_groups.merge(int_groups_tot, on=['Product type', 'Region', 'COUNTRY/TERRITORY', 'Product group', 'PRODUCT', 'Gas'], how='left')
cols_x = [col for col in merged.columns if col.endswith('_x')][:-1]
cols_y = [col for col in merged.columns if col.endswith('_y')][:-1]
merged[cols_x] = merged[cols_x].div(merged[cols_y].values).fillna(0)
merged.columns = [i.replace('_x','') for i in merged.columns]
merged = merged.drop(merged.filter(like='_y').columns, axis=1)

merged = merged.merge(rm_groups, on=['Product type', 'Region', 'COUNTRY/TERRITORY', 'Product group', 'PRODUCT', 'Gas'], how='left')
cols_x = [col for col in merged.columns if col.endswith('_x')][1:-1]
cols_y = [col for col in merged.columns if col.endswith('_y')][1:-1]
merged[cols_x] = merged[cols_x].multiply(merged[cols_y].values).fillna(0)
merged.columns = [i.replace('_x','') for i in merged.columns]
merged = merged.drop(merged.filter(like='_y').columns, axis=1)

grouped_filt = pd.concat((grouped[[i not in ['Feedstock', 'Organic chemicals', 'Primary chemicals', 'Other intermediates'] for i in grouped['Type']]], merged))

In [4]:
# convert regional other to other + export
grouped_filt['Product group'] = grouped_filt['Product group'].replace({'Other downstream': 'Other downstream & exports'})

# Make worldwide values
world = grouped_filt.groupby(['Product type', 'Product group', 'PRODUCT', 'Gas', 'Type']).sum().reset_index()
world['Region'], world['COUNTRY/TERRITORY'] = 'World', 'World'
world['Product group'] = world['Product group'].replace({'Other downstream & exports': 'Other downstream'})
flow_df = pd.concat((grouped_filt, world))

In [5]:
# Re-inflate Ammonia uncertainties after artificial collapse in setting productino weighting
uncert = 0.15
year_cols = [str(i) for i in range(1978,2051)]
sigma_cols = [str(i)+'_sigma' for i in range(1978,2051)]

amm_exerp = flow_df[flow_df['PRODUCT']=='AMMONIA']
amm_exerp[sigma_cols] = amm_exerp[year_cols].values*uncert
flow_df = pd.concat((flow_df[flow_df['PRODUCT']!='AMMONIA'], amm_exerp))

C:\Users\lukec\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [6]:
# Surpress intragroup dependencies
sankey_table = flow_df.copy()
surpressed = flow_df[flow_df['Product group']!=flow_df['Type']]
surpressed_nonfert = surpressed[(surpressed['PRODUCT']!='AMMONIA') & (surpressed['Product group']!='N-fertilisers')]
surpressed_amm = pd.concat((surpressed[(surpressed['PRODUCT']=='AMMONIA')], surpressed[(surpressed['Product group']=='N-fertilisers')]))

year_cols = [str(i) for i in range(1978,2051)]
sigma_cols = [str(i)+'_sigma' for i in range(1978,2051)]
group_by1, group_by2 = ['Product group', 'Gas'], ['Type', 'Gas']

for region in tqdm(flow_df['Region'].unique()):
    # Account for unallocated intermediates
    int_cats = ['Primary chemicals', 'Organic chemicals', 'Other intermediates']
    types = ['Direct Energy Use', 'Indirect Energy Use', 'Feedstock', 'Direct Process']
    ints = filter_df(surpressed_nonfert, ['Region', 'Product group', 'Type'], [[region], int_cats, types+int_cats]).groupby(group_by1).sum()
    ints_used = filter_df(surpressed_nonfert, ['Region', 'Type', 'Type'], [[region], int_cats, types+int_cats]).groupby(group_by2).sum()
    diff = (ints-ints_used).reset_index().rename(columns={'Product group':'Type'})
    diff[sigma_cols] = (abs((ints[year_cols]-ints_used[year_cols]).values/ints[year_cols]).values*ints[sigma_cols]).values
    diff['Region'], diff['Product group'], diff['Product type'] = region, 'Other downstream', 'Downstream'
    unalloc = diff.copy()
    unalloc[year_cols+sigma_cols] = diff[year_cols+sigma_cols].clip(0).values

    # Account for unallocated ammonia to fertilisers
    ints_amm = filter_df(surpressed_amm, ['Region', 'Product group', 'Type'], [[region], 'Primary chemicals', types+int_cats]).groupby(group_by1).sum()
    ints_used_amm = filter_df(surpressed_amm, ['Region', 'Type', 'Type'], [[region], 'Primary chemicals', types+int_cats]).groupby(group_by2).sum()
    diff_amm = (ints_amm-ints_used_amm).reset_index().rename(columns={'Product group':'Type'})
    diff_amm[sigma_cols] = (abs((ints_amm[year_cols]-ints_used_amm[year_cols]).values/ints_amm[year_cols]).values*ints_amm[sigma_cols]).values
    diff_amm['Region'], diff_amm['Product group'], diff_amm['PRODUCT'], diff_amm['Product type'] = region, 'N-fertilisers', 'AMMONIA', 'Downstream'
    unalloc_amm = diff_amm.copy()
    unalloc_amm[year_cols+sigma_cols] = diff_amm[year_cols+sigma_cols].clip(0).values

    # Imports/exports if regional
    if region!='World':
        imports = diff.copy()
        imports[year_cols+sigma_cols] = imports[year_cols+sigma_cols].clip(upper=0).values
        imports['Region'], imports['Product group'], imports['Type'], imports[year_cols+sigma_cols] = region, imports['Type'], 'Imports', abs(imports[year_cols+sigma_cols])
        unalloc['Product group'] = unalloc['Product group'].replace({'Other downstream':'Other downstream & exports'})
        imports['Product group'] = imports['Product group'].replace({'Other downstream':'Other downstream & exports'})
        sankey_table = pd.concat((sankey_table, unalloc, unalloc_amm, imports))
    else: sankey_table = pd.concat((sankey_table, unalloc, unalloc_amm))

100%|██████████| 10/10 [00:05<00:00,  1.87it/s]


In [96]:
sankey_table

,Region,COUNTRY/TERRITORY,Product group,PRODUCT,Gas,Type,1978,1979,1980,1981,...,2042_sigma,2043_sigma,2044_sigma,2045_sigma,2046_sigma,2047_sigma,2048_sigma,2049_sigma,2050_sigma,Product type
0,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,Direct Energy Use,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Downstream
1,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,Direct Process,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Downstream
2,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,EI & CM,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Downstream
3,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,Electricity,3.607483e+01,3.807786e+01,4.048651e+01,4.057639e+01,...,2.130371e+00,2.142740e+00,2.155109e+00,2.167479e+00,2.179848e+00,2.192217e+00,2.204586e+00,2.216955e+00,2.229325e+00,Downstream
5,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,IHS CtOG,9.045944e+02,9.548214e+02,1.015219e+03,1.017473e+03,...,5.342012e+01,5.373029e+01,5.404045e+01,5.435061e+01,5.466078e+01,5.497094e+01,5.528110e+01,5.559127e+01,5.590143e+01,Downstream
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,World,NaN,N-fertilisers,AMMONIA,Nitric oxide,Primary chemicals,4.364182e-07,4.606500e-07,4.897888e-07,4.908762e-07,...,1.645645e-07,1.655199e-07,1.664754e-07,1.674309e-07,1.683864e-07,1.693419e-07,1.702973e-07,1.712528e-07,1.722083e-07,Downstream
9,World,NaN,N-fertilisers,AMMONIA,Nitrogen fluoride,Primary chemicals,7.081856e-11,7.475071e-11,7.947914e-11,7.965559e-11,...,2.670424e-11,2.685929e-11,2.701434e-11,2.716939e-11,2.732444e-11,2.747948e-11,2.763453e-11,2.778958e-11,2.794463e-11,Downstream
10,World,NaN,N-fertilisers,AMMONIA,Other,Primary chemicals,4.883999e-06,5.155179e-06,5.481275e-06,5.493444e-06,...,1.841657e-06,1.852350e-06,1.863043e-06,1.873736e-06,1.884429e-06,1.895121e-06,1.905814e-06,1.916507e-06,1.927200e-06,Downstream
11,World,NaN,N-fertilisers,AMMONIA,Perfluoropentane,Primary chemicals,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Downstream


In [7]:
## Propagate uncertainties from primary chems to downstream
element = 'Primary chemicals'
grouped_ints = filter_df(sankey_table, ['Product group', 'Type'], [element, types+int_cats]).groupby(['Product group', 'Gas']).sum()
int_error_ratio = grouped_ints[sigma_cols].values/grouped_ints[year_cols].values
grouped_ints[year_cols] = int_error_ratio
error_ratio_df = grouped_ints[year_cols].reset_index().rename(columns={'Product group':'Type'})

ds_from_ints = filter_df(sankey_table, ['Type'], [element])
ds_from_ints = ds_from_ints[ds_from_ints['Product group']!=element]
merged = ds_from_ints.merge(error_ratio_df, on=['Type', 'Gas'], how='left', suffixes=('', '_y'))
merged[sigma_cols] = merged[year_cols].values*merged[[i+'_y' for i in year_cols]].values
merged = merged.drop(columns=[i+'_y' for i in year_cols])

table_corrected_prim = pd.concat((sankey_table[[(source_type!=element) or (prod_type == element) for (source_type, prod_type) in zip(sankey_table['Type'], sankey_table['Product group'])]],
                             merged))

C:\Users\lukec\AppData\Local\Temp/ipykernel_4172/1259213820.py:4: RuntimeWarning: invalid value encountered in true_divide
  int_error_ratio = grouped_ints[sigma_cols].values/grouped_ints[year_cols].values


In [8]:
table_corrected_prim

,Region,COUNTRY/TERRITORY,Product group,PRODUCT,Gas,Type,1978,1979,1980,1981,...,2042_sigma,2043_sigma,2044_sigma,2045_sigma,2046_sigma,2047_sigma,2048_sigma,2049_sigma,2050_sigma,Product type
0,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,Direct Energy Use,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Downstream
1,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,Direct Process,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Downstream
2,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,EI & CM,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Downstream
3,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,Electricity,3.607483e+01,3.807786e+01,4.048651e+01,4.057639e+01,...,2.130371e+00,2.142740e+00,2.155109e+00,2.167479e+00,2.179848e+00,2.192217e+00,2.204586e+00,2.216955e+00,2.229325e+00,Downstream
5,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,IHS CtOG,9.045944e+02,9.548214e+02,1.015219e+03,1.017473e+03,...,5.342012e+01,5.373029e+01,5.404045e+01,5.435061e+01,5.466078e+01,5.497094e+01,5.528110e+01,5.559127e+01,5.590143e+01,Downstream
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30012,World,NaN,N-fertilisers,AMMONIA,Nitric oxide,Primary chemicals,4.364182e-07,4.606500e-07,4.897888e-07,4.908762e-07,...,7.760199e-07,7.824080e-07,7.897063e-07,7.945550e-07,8.002566e-07,8.057371e-07,8.111347e-07,8.173101e-07,1.316919e-06,Downstream
30013,World,NaN,N-fertilisers,AMMONIA,Nitrogen fluoride,Primary chemicals,7.081856e-11,7.475071e-11,7.947914e-11,7.965559e-11,...,1.125194e-10,1.137106e-10,1.149564e-10,1.158226e-10,1.167103e-10,1.177151e-10,1.185978e-10,1.196187e-10,2.662476e-10,Downstream
30014,World,NaN,N-fertilisers,AMMONIA,Other,Primary chemicals,4.883999e-06,5.155179e-06,5.481275e-06,5.493444e-06,...,7.591784e-06,7.654378e-06,7.723847e-06,7.769716e-06,7.825563e-06,7.879818e-06,7.932649e-06,7.994174e-06,1.410339e-05,Downstream
30015,World,NaN,N-fertilisers,AMMONIA,Perfluoropentane,Primary chemicals,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Downstream


In [9]:
## Propagate uncertainties from organic chems to downstream
element = 'Organic chemicals'
grouped_ints = filter_df(table_corrected_prim, ['Product group', 'Type'], [element, types+int_cats]).groupby(['Product group', 'Gas']).sum()
int_error_ratio = grouped_ints[sigma_cols].values/grouped_ints[year_cols].values
grouped_ints[year_cols] = int_error_ratio
error_ratio_df = grouped_ints[year_cols].reset_index().rename(columns={'Product group':'Type'})

ds_from_ints = filter_df(table_corrected_prim, ['Type'], [element])
ds_from_ints = ds_from_ints[ds_from_ints['Product group'] != element]
merged = ds_from_ints.merge(error_ratio_df, on=['Type', 'Gas'], how='left', suffixes=('', '_y'))
merged[sigma_cols] = merged[year_cols].values*merged[[i+'_y' for i in year_cols]].values
merged = merged.drop(columns=[i+'_y' for i in year_cols])

table_corrected_org = pd.concat((table_corrected_prim[[(source_type not in [element]) or (prod_type in [element]) for (source_type, prod_type) in zip(table_corrected_prim['Type'], table_corrected_prim['Product group'])]], merged))

C:\Users\lukec\AppData\Local\Temp/ipykernel_4172/2820217800.py:4: RuntimeWarning: invalid value encountered in true_divide
  int_error_ratio = grouped_ints[sigma_cols].values/grouped_ints[year_cols].values


In [10]:
table_corrected_org

,Region,COUNTRY/TERRITORY,Product group,PRODUCT,Gas,Type,1978,1979,1980,1981,...,2042_sigma,2043_sigma,2044_sigma,2045_sigma,2046_sigma,2047_sigma,2048_sigma,2049_sigma,2050_sigma,Product type
0,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,Direct Energy Use,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Downstream
1,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,Direct Process,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Downstream
2,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,EI & CM,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Downstream
3,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,Electricity,36.074828,38.077858,40.486509,40.576394,...,2.130371,2.142740,2.155109,2.167479,2.179848,2.192217,2.204586,2.216955,2.229325,Downstream
5,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,IHS CtOG,904.594387,954.821355,1015.219485,1017.473406,...,53.420123,53.730286,54.040450,54.350613,54.660777,54.970940,55.281104,55.591267,55.901431,Downstream
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24708,World,NaN,Other downstream,NaN,Nitric oxide,Organic chemicals,431.120794,429.263970,337.869314,352.269679,...,71.901761,72.070505,72.286948,72.456312,72.656924,72.826372,73.027124,73.197241,73.397731,Downstream
24709,World,NaN,Other downstream,NaN,Nitrogen fluoride,Organic chemicals,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Downstream
24710,World,NaN,Other downstream,NaN,Other,Organic chemicals,0.001965,0.002083,0.001683,0.001711,...,0.088959,0.090086,0.091282,0.092425,0.093569,0.094721,0.095892,0.097035,0.098206,Downstream
24711,World,NaN,Other downstream,NaN,Perfluoropentane,Organic chemicals,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Downstream


In [11]:
## Propagate uncertainties from other intermediates to downstream
element = 'Other intermediates'
grouped_ints = filter_df(table_corrected_org, ['Product group', 'Type'], [element, types+int_cats]).groupby(['Product group', 'Gas']).sum()
int_error_ratio = grouped_ints[sigma_cols].values/grouped_ints[year_cols].values
grouped_ints[year_cols] = int_error_ratio
error_ratio_df = grouped_ints[year_cols].reset_index().rename(columns={'Product group':'Type'})

ds_from_ints = filter_df(table_corrected_org, ['Type'], [element])
ds_from_ints = ds_from_ints[ds_from_ints['Product group'] != element]
merged = ds_from_ints.merge(error_ratio_df, on=['Type', 'Gas'], how='left', suffixes=('', '_y'))
merged[sigma_cols] = merged[year_cols].values*merged[[i+'_y' for i in year_cols]].values
merged = merged.drop(columns=[i+'_y' for i in year_cols])

table_corrected_final = pd.concat((table_corrected_org[[(source_type not in [element]) or (prod_type in [element]) for (source_type, prod_type) in zip(table_corrected_org['Type'], table_corrected_org['Product group'])]], merged))

C:\Users\lukec\AppData\Local\Temp/ipykernel_4172/2577686036.py:4: RuntimeWarning: invalid value encountered in true_divide
  int_error_ratio = grouped_ints[sigma_cols].values/grouped_ints[year_cols].values


In [12]:
table_corrected_final

,Region,COUNTRY/TERRITORY,Product group,PRODUCT,Gas,Type,1978,1979,1980,1981,...,2042_sigma,2043_sigma,2044_sigma,2045_sigma,2046_sigma,2047_sigma,2048_sigma,2049_sigma,2050_sigma,Product type
0,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,Direct Energy Use,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Downstream
1,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,Direct Process,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Downstream
2,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,EI & CM,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Downstream
3,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,Electricity,3.607483e+01,3.807786e+01,4.048651e+01,4.057639e+01,...,2.130371e+00,2.142740e+00,2.155109e+00,2.167479e+00,2.179848e+00,2.192217e+00,2.204586e+00,2.216955e+00,2.229325e+00,Downstream
5,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,IHS CtOG,9.045944e+02,9.548214e+02,1.015219e+03,1.017473e+03,...,5.342012e+01,5.373029e+01,5.404045e+01,5.435061e+01,5.466078e+01,5.497094e+01,5.528110e+01,5.559127e+01,5.590143e+01,Downstream
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35017,World,NaN,Other downstream,NaN,Nitric oxide,Other intermediates,4.882294e+00,5.288488e+00,4.866365e+00,4.898223e+00,...,5.145563e+01,5.207641e+01,5.283514e+01,5.359387e+01,5.428362e+01,5.497338e+01,5.573211e+01,5.655981e+01,5.731854e+01,Downstream
35018,World,NaN,Other downstream,NaN,Nitrogen fluoride,Other intermediates,4.425370e-11,4.926674e-11,4.262703e-11,4.477551e-11,...,8.698617e-11,8.928579e-11,9.174815e-11,9.390048e-11,9.606526e-11,9.841274e-11,1.006254e-10,1.030609e-10,1.512614e-10,Downstream
35019,World,NaN,Other downstream,NaN,Other,Other intermediates,3.097807e-06,3.367378e-06,3.033026e-06,3.051257e-06,...,1.942546e-05,1.977487e-05,2.020604e-05,2.061973e-05,2.098590e-05,2.137985e-05,2.179723e-05,2.225909e-05,2.268550e-05,Downstream
35020,World,NaN,Other downstream,NaN,Perfluoropentane,Other intermediates,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Downstream


In [14]:
## Compensate for Other intermediates which are used more than are known to be produced

used_types = filter_df(table_corrected_final, ['Type'], [['Imports', 'Indirect Energy Use', 'Direct Energy Use', 'Feedstock', 'Primary chemicals', 'Organic chemicals', 'Other intermediates', 'Direct Process']])

multiply_types = filter_df(used_types, ['Type'], [['Other intermediates']]).groupby(['Region', 'Gas']).sum()-filter_df(used_types, ['Product group'], [['Other intermediates']]).groupby(['Region','Gas']).sum()

oi_compensation = multiply_types.clip(0).reset_index()
oi_compensation['Product type'], oi_compensation['Product group'], oi_compensation['PRODUCT'], oi_compensation['Type'] = 'Intermediates', 'Other intermediates', 'PROPYLENE GLYCOL', 'Feedstock'
oi_compensation['COUNTRY/TERRITORY'] = oi_compensation['Region']

table_compensated = pd.concat((table_corrected_final, oi_compensation)).sort_values(['Region', 'COUNTRY/TERRITORY', 'Product group', 'PRODUCT'])

table_compensated.to_parquet('C:/Users\lukec\OneDrive - University of Cambridge\PhD\Power_BI\input_data/country_summaries_w_uncertainties.parquet')

In [15]:
# Unpivot data so there is a row per product/per year for easy filtering in Power BI
year_cols = [str(i) for i in range(1978,2051)]
sigma_cols = [str(i)+'_sigma' for i in range(1978,2051)]
id_cols = ['Region', 'COUNTRY/TERRITORY', 'Product type', 'Product group', 'PRODUCT', 'Gas', 'Type']
melted_vals = pd.melt(table_compensated[id_cols+year_cols], id_vars=id_cols, var_name='Year', value_name='y')
melted_errors = pd.melt(table_compensated[id_cols+sigma_cols], id_vars=id_cols, var_name='Year', value_name='sigma_y')
melted_vals['sigma_y'] = melted_errors['sigma_y']
melted_vals['Year'] = melted_vals['Year'].astype(int)

In [16]:
melted_vals.to_parquet('C:/Users\lukec\OneDrive - University of Cambridge\PhD\Power_BI\input_data/country_summaries_w_uncertainties_unpivoted.parquet')
#melted_vals.to_parquet('C:/Users\lukec\OneDrive - University of Cambridge\PhD\Power_BI\input_data/country_summaries_unpivoted.parquet')

In [ ]:
country_regions = pd.read_parquet('C:/Users\lukec\OneDrive - University of Cambridge\PhD\Power_BI\input_data/country_summaries_unpivoted.parquet')

## Even out emissions

In [8]:
subset = filter_df(table_corrected, ['Gas', 'Type', 'Region'], [['CO2e_100a'], ['Imports', 'Indirect Energy Use', 'Direct Energy Use', 'Feedstock', 'Primary chemicals', 'Organic chemicals', 'Other intermediates', 'Direct Process'], ['World']])

In [17]:
subset.groupby(['Product group']).sum()['2020']

Product group
N-fertilisers                       5.353519e+05
Organic chemicals                   9.603597e+05
Other downstream                    2.747002e+05
Other intermediates                 1.940518e+04
Primary chemicals                   1.113601e+06
Solvents, additives & explosives    1.457152e+04
Thermoplastics                      8.744034e+05
Thermosets, fibre & elastomers      4.361340e+05
Name: 2020, dtype: float64

In [49]:
table_compensated

,Region,COUNTRY/TERRITORY,Product group,PRODUCT,Gas,Type,1978,1979,1980,1981,...,2042_sigma,2043_sigma,2044_sigma,2045_sigma,2046_sigma,2047_sigma,2048_sigma,2049_sigma,2050_sigma,Product type
0,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,Direct Energy Use,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Downstream
1,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,Direct Process,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Downstream
2,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,EI & CM,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Downstream
3,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,Electricity,36.074828,38.077858,40.486509,40.576394,...,2.130371,2.142740,2.155109,2.167479,2.179848,2.192217,2.204586,2.216955,2.229325,Downstream
5,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,IHS CtOG,904.594387,954.821355,1015.219485,1017.473406,...,53.420123,53.730286,54.040450,54.350613,54.660777,54.970940,55.281104,55.591267,55.901431,Downstream
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50968,World,None,Other downstream,None,Nitric oxide,Primary chemicals,0.000027,0.000030,0.000028,0.000028,...,0.000102,0.000103,0.000104,0.000105,0.000106,0.000107,0.000108,0.000109,0.000177,Downstream
50969,World,None,Other downstream,None,Nitrogen fluoride,Primary chemicals,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Downstream
50970,World,None,Other downstream,None,Other,Primary chemicals,0.000067,0.000074,0.000070,0.000069,...,0.000236,0.000239,0.000242,0.000244,0.000247,0.000249,0.000251,0.000254,0.000449,Downstream
50971,World,None,Other downstream,None,Perfluoropentane,Primary chemicals,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Downstream


In [46]:

oi_compensation


,Region,Gas,1978,1979,1980,1981,1982,1983,1984,1985,...,2045_sigma,2046_sigma,2047_sigma,2048_sigma,2049_sigma,2050_sigma,Product group,PRODUCT,Type,COUNTRY/TERRITORY
0,Africa,CO2e_100a,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,1.848775e+02,1.862930e+02,1.877369e+02,1.891072e+02,1.906081e+02,206.251681,Other intermediates,PROPYLENE GLYCOL,Feedstock,Africa
1,Africa,CO2e_20a,0.0,0.000000e+00,7.105427e-15,0.0,0.0,0.0,0.0,0.0,...,2.454252e+02,2.473130e+02,2.492206e+02,2.510313e+02,2.530124e+02,274.238155,Other intermediates,PROPYLENE GLYCOL,Feedstock,Africa
2,Africa,Carbon dioxide,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,1.645126e+02,1.657730e+02,1.670629e+02,1.682864e+02,1.696246e+02,181.523352,Other intermediates,PROPYLENE GLYCOL,Feedstock,Africa
3,Africa,Carbon monoxide,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,7.175800e-01,7.231148e-01,7.279355e-01,7.336404e-01,7.391492e-01,0.806363,Other intermediates,PROPYLENE GLYCOL,Feedstock,Africa
4,Africa,Chloroform,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,Other intermediates,PROPYLENE GLYCOL,Feedstock,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,World,Nitric oxide,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,7.105427e-15,0.000000e+00,0.000000e+00,7.105427e-15,7.105427e-15,0.000000,Other intermediates,PROPYLENE GLYCOL,Feedstock,World
126,World,Nitrogen fluoride,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,1.292470e-26,0.000000e+00,1.292470e-26,0.000000e+00,0.000000,Other intermediates,PROPYLENE GLYCOL,Feedstock,World
127,World,Other,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,6.776264e-21,0.000000e+00,6.776264e-21,0.000000,Other intermediates,PROPYLENE GLYCOL,Feedstock,World
128,World,Perfluoropentane,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,Other intermediates,PROPYLENE GLYCOL,Feedstock,World


In [13]:
subset.groupby('Product group').sum()['2020']

Product group
N-fertilisers                       5.353519e+05
Organic chemicals                   9.603597e+05
Other downstream                    2.747002e+05
Other intermediates                 1.940518e+04
Primary chemicals                   1.113601e+06
Solvents, additives & explosives    1.457152e+04
Thermoplastics                      8.744034e+05
Thermosets, fibre & elastomers      4.361340e+05
Name: 2020, dtype: float64

## Get correct emissions

In [ ]:
if region=='world':
        region_data = data.copy()
        dp_negs_reg = neg_dps.copy()
        dp_pos_reg = pos_dps.copy()
    else:
        region_data = filter_df(data, ['Region'], [region])
        dp_negs_reg = filter_df(neg_dps, ['Region'], [region])
        dp_pos_reg = filter_df(pos_dps, ['Region'], [region])
    reg_sum = region_data.groupby(['Product type', 'Product group', 'Type']).sum().reset_index().rename(columns={'Product type':'Level', 'Type': 'source', 'PRODUCT':'Product',
                                                                                                                 'y':'value', 'sigma_y':'error', 'Product group':'target'})
    reg_sum['type'] = reg_sum['source']
    reg_sum=reg_sum[reg_sum['type']!='Direct Process']
    sankey_table = pd.concat((reg_sum[reg_sum['target']!=reg_sum['source']], dp_negs_reg, dp_pos_reg))

    # Account for unallocated intermediates
    int_cats = ['Primary chemicals', 'Organic chemicals', 'Other intermediates']
    ints = filter_df(sankey_table, ['target'], [int_cats]).groupby('target').sum()
    ints_used = filter_df(sankey_table, ['source'], [int_cats]).groupby('source').sum()
    diff = (ints-ints_used).reset_index().rename(columns={'target':'source'})
    diff['error'] = (abs((ints['value']-ints_used['value']).values/ints['value'])*ints['error']).values
    diff['target'], diff['type'], diff['Level'] = 'Other downstream', diff['source'], 'Downstream'
    unalloc = diff[diff['value'] > 0]

    # Imports/exports if regional
    downstream_order_used = downstream_order.copy()
    if region!='world':
        imports = diff[diff['value'] < 0]
        imports['target'], imports['source'], imports['value'] = imports['source'], 'Imports', abs(imports['value'])
        sankey_table = pd.concat((sankey_table, unalloc, imports))

        sankey_table['target'] = sankey_table['target'].replace({'Other downstream':'Other downstream & exports'})
        downstream_order_used[-1] = 'Other downstream & exports'
    else: sankey_table = pd.concat((sankey_table, unalloc))

## Get emissions for subsets

In [2]:
pivoted = pd.read_parquet('C:/Users\lukec\OneDrive - University of Cambridge\PhD\Power_BI\input_data/countries_uncertainties_unpivoted_allgases_final2.parquet')

In [10]:
year = 2020
#country = 'United Kingdom'
#region = 'North East Asia'
lc_type = 'Cradle-to-out-gate'
gases = ['CO2e_100a']
#product = 'AMMONIA'
prod_type = 'Downstream'
#fert_type = ['Fertiliser']

def filt_df(df, column, filt):
    if type(filt) is not list:
        filt = [filt]
    return df[[i in filt for i in df[column]]]

df = pivoted.copy()
for col, filt in zip(['Year', 'Product type', 'Gas', 'Type'], [year, prod_type, gases, lc_type]):
    df = filt_df(df, col, filt)
df

Measure,Region,COUNTRY/TERRITORY,Product group,PRODUCT,Gas,Type,Product type,Fertiliser type,Year,sigma_y,y
42,Africa,Africa,N-fertilisers,AMMONIA,CO2e_100a,Cradle-to-out-gate,Downstream,Fertiliser,2020,1830.987460,9866.339509
6685,Africa,Africa,N-fertilisers,AMMONIUM NITRATE,CO2e_100a,Cradle-to-out-gate,Downstream,Fertiliser,2020,35.415609,1266.767145
13328,Africa,Africa,N-fertilisers,AMMONIUM SULPHATE,CO2e_100a,Cradle-to-out-gate,Downstream,Fertiliser,2020,5.494049,262.227752
19971,Africa,Africa,N-fertilisers,CALCIUM AMMONIUM NITRATE,CO2e_100a,Cradle-to-out-gate,Downstream,Fertiliser,2020,2.732253,97.728910
26614,Africa,Africa,N-fertilisers,UREA,CO2e_100a,Cradle-to-out-gate,Downstream,Fertiliser,2020,447.668750,8904.554315
...,...,...,...,...,...,...,...,...,...,...,...
17929499,South Asia,Vietnam,"Thermosets, fibre & elastomers",POLYAMIDE FIBRES,CO2e_100a,Cradle-to-out-gate,Downstream,Non-fertiliser,2020,14.133027,774.829678
17936142,South Asia,Vietnam,"Thermosets, fibre & elastomers",POLYBUTADIENE,CO2e_100a,Cradle-to-out-gate,Downstream,Non-fertiliser,2020,0.000000,0.000000
17942785,South Asia,Vietnam,"Thermosets, fibre & elastomers",POLYESTER FIBRES,CO2e_100a,Cradle-to-out-gate,Downstream,Non-fertiliser,2020,20.665144,1213.618647
17949428,South Asia,Vietnam,"Thermosets, fibre & elastomers",POLYMETHYL METHACRYLATE,CO2e_100a,Cradle-to-out-gate,Downstream,Non-fertiliser,2020,4.462867,184.628220


In [16]:
proportion = df.groupby(['Region', 'Product group']).sum().reset_index().merge(df.groupby(['Region']).sum().reset_index(), on='Region')
proportion['Percent'] = proportion['y_x']/proportion['y_y']
proportion

Measure,Region,Product group,Year_x,sigma_y_x,y_x,Year_y,sigma_y_y,y_y,Percent
0,Africa,N-fertilisers,10100,2322.298121,20397.617631,129280,2990.845931,29075.472652,0.701540
1,Africa,Other downstream,4040,1.419561,29.075884,129280,2990.845931,29075.472652,0.001000
2,Africa,"Solvents, additives & explosives",18180,178.196669,470.271596,129280,2990.845931,29075.472652,0.016174
3,Africa,Thermoplastics,62620,470.724571,7698.959446,129280,2990.845931,29075.472652,0.264792
4,Africa,"Thermosets, fibre & elastomers",34340,18.207009,479.548095,129280,2990.845931,29075.472652,0.016493
5,Europe,N-fertilisers,10100,3819.619084,36735.386128,896880,10612.990680,158222.353509,0.232176
6,Europe,Other downstream,78780,154.553456,2343.258578,896880,10612.990680,158222.353509,0.014810
7,Europe,"Solvents, additives & explosives",121200,445.663750,1894.375908,896880,10612.990680,158222.353509,0.011973
8,Europe,Thermoplastics,408040,5318.998223,85978.731484,896880,10612.990680,158222.353509,0.543404
9,Europe,"Thermosets, fibre & elastomers",278760,874.156167,31270.601411,896880,10612.990680,158222.353509,0.197637


In [13]:
year = 2019
country = 'United Kingdom'
lc_type = 'Cradle-to-out-gate'
gases = ['CO2e_100a', 'CO2e_20a', 'Carbon dioxide', 'Methane']

def filt_df(df, column, filt):
    if type(filt) is not list:
        filt = [filt]
    return df[[i in filt for i in df[column]]]

df = pivoted.copy()
for col, filt in zip(['Year', 'COUNTRY/TERRITORY', 'Type', 'Gas'], [year, country, lc_type, gases]):
    df = filt_df(df, col, filt)

In [16]:
df.reset_index(drop=True).drop(columns=['Fertiliser type']).rename(columns={'Measure':'index', 'Type':'Life-cycle', 'sigma_y':'Uncertainty', 'y': 'Emissions (kt)'}).to_csv('C:/Users\lukec\PycharmProjects\petrochemical-data/analysis_outputs/requested_outputs/UK_2019_CtOG_emissions.csv')

Measure,Region,COUNTRY/TERRITORY,Product group,PRODUCT,Gas,Type,Product type,Fertiliser type,Year,sigma_y,y
6719764,Europe,United Kingdom,Organic chemicals,2-ETHYLHEXANOL,CO2e_100a,Cradle-to-out-gate,Intermediates,Non-fertiliser,2019,0.000000,0.000000
6720275,Europe,United Kingdom,Organic chemicals,2-ETHYLHEXANOL,CO2e_20a,Cradle-to-out-gate,Intermediates,Non-fertiliser,2019,0.000000,0.000000
6720786,Europe,United Kingdom,Organic chemicals,2-ETHYLHEXANOL,Carbon dioxide,Cradle-to-out-gate,Intermediates,Non-fertiliser,2019,0.000000,0.000000
6723341,Europe,United Kingdom,Organic chemicals,2-ETHYLHEXANOL,Methane,Cradle-to-out-gate,Intermediates,Non-fertiliser,2019,0.000000,0.000000
6726407,Europe,United Kingdom,Organic chemicals,ACETALDEHYDE,CO2e_100a,Cradle-to-out-gate,Intermediates,Non-fertiliser,2019,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
7175065,Europe,United Kingdom,"Thermosets, fibre & elastomers",SBR (Includes SBCs),Methane,Cradle-to-out-gate,Downstream,Non-fertiliser,2019,0.012017,0.342621
7178131,Europe,United Kingdom,"Thermosets, fibre & elastomers",VINYL ACETATE M.,CO2e_100a,Cradle-to-out-gate,Downstream,Non-fertiliser,2019,0.000000,0.000000
7178642,Europe,United Kingdom,"Thermosets, fibre & elastomers",VINYL ACETATE M.,CO2e_20a,Cradle-to-out-gate,Downstream,Non-fertiliser,2019,0.000000,0.000000
7179153,Europe,United Kingdom,"Thermosets, fibre & elastomers",VINYL ACETATE M.,Carbon dioxide,Cradle-to-out-gate,Downstream,Non-fertiliser,2019,0.000000,0.000000


## Paper plotting

In [5]:
group_props = ['Region', 'Product group', 'PRODUCT', 'Gas', 'Type','Product type']

per_area = grouped[group_props+['2020']].groupby(group_props).sum().reset_index()

NameError: name 'grouped' is not defined

In [75]:
# country_groupings['COUNTRY/TERRITORY'] = country_groupings['COUNTRY/TERRITORY'].str.title().replace(country_replace_dict)
# area_summary_plots = country_groupings.merge(per_area, how='outer', on='Region')

,Region,Product group,PRODUCT,Gas,Type,Product type,2020
0,Africa,Organic chemicals,ACETALDEHYDE,CO2e_100a,Cradle-to-out-gate,Intermediates,0.000000
1,Africa,Organic chemicals,ACETALDEHYDE,CO2e_100a,Direct Process,Intermediates,0.000000
2,Africa,Organic chemicals,ACETALDEHYDE,CO2e_100a,Direct Utilities,Intermediates,0.000000
3,Africa,Organic chemicals,ACETALDEHYDE,CO2e_100a,EI & CM,Intermediates,0.000000
4,Africa,Organic chemicals,ACETALDEHYDE,CO2e_100a,Electricity,Intermediates,0.000000
...,...,...,...,...,...,...,...
65970,South Asia,"Thermosets, fibre & elastomers",VINYL ACETATE M.,Sulfur hexafluoride,Direct Utilities,Downstream,0.000000
65971,South Asia,"Thermosets, fibre & elastomers",VINYL ACETATE M.,Sulfur hexafluoride,EI & CM,Downstream,0.000133
65972,South Asia,"Thermosets, fibre & elastomers",VINYL ACETATE M.,Sulfur hexafluoride,Electricity,Downstream,0.000004
65973,South Asia,"Thermosets, fibre & elastomers",VINYL ACETATE M.,Sulfur hexafluoride,Indirect Utilities,Downstream,0.000007


In [76]:
per_area.reset_index().to_parquet(data_path+'input_data/area_summary_plots.parquet', index=False)

In [27]:
country_groupings.to_csv(data_path+'input_data/tileset_demo.csv')

In [12]:
import geopandas as gpd
country_jsons = gpd.read_file("C:/Users\lukec\Downloads\countries.geojson")

country_shapes = country_jsons.rename(columns={'ADMIN':'COUNTRY/TERRITORY'}).merge(country_groupings, how='left', on='COUNTRY/TERRITORY')

In [18]:
region_allocation = pd.read_csv(data_path+'extra_inputs/country_shapes.csv', index_col='Unnamed: 0')

final_country_shapes = country_shapes[country_shapes.columns[:-1]].merge(region_allocation, how='left', on='COUNTRY/TERRITORY').dropna()

final_regions = gpd.GeoDataFrame(final_country_shapes).dissolve('Region')

final_regions = final_regions.reset_index().drop(columns=['COUNTRY/TERRITORY', 'ISO_A3'])

final_regions.geometry = final_regions.geometry.simplify(tolerance=0.01)

final_regions.to_file(data_path+'extra_inputs/country_shapes_simp.geojson', driver='GeoJSON')

In [16]:
final_regions.geometry

0    MULTIPOLYGON (((-3.11531 5.10781, -3.11528 5.1...
1    MULTIPOLYGON (((-53.71458 5.67441, -53.69560 5...
2    MULTIPOLYGON (((45.62779 38.95550, 45.57084 38...
3    MULTIPOLYGON (((-109.27392 -27.13567, -109.292...
4    MULTIPOLYGON (((42.71925 13.74262, 42.76173 13...
5    MULTIPOLYGON (((-176.47932 0.19033, -176.48127...
6    MULTIPOLYGON (((111.20753 15.78217, 111.20460 ...
7    MULTIPOLYGON (((-176.22566 -44.29860, -176.212...
8    MULTIPOLYGON (((72.48463 -7.37591, 72.48414 -7...
Name: geometry, dtype: geometry

0    MULTIPOLYGON (((-3.11531 5.10781, -3.09899 5.1...
1    MULTIPOLYGON (((-53.71458 5.67441, -53.69560 5...
2    MULTIPOLYGON (((45.62779 38.95550, 45.57084 38...
3    MULTIPOLYGON (((-109.27392 -27.13567, -109.292...
4    MULTIPOLYGON (((42.71925 13.74262, 42.76173 13...
5    MULTIPOLYGON (((-176.47932 0.19033, -176.48127...
6    MULTIPOLYGON (((111.20753 15.78217, 111.19654 ...
7    MULTIPOLYGON (((-176.22566 -44.29860, -176.212...
8    MULTIPOLYGON (((72.48463 -7.37591, 72.48414 -7...
dtype: geometry